In [ ]:

# ## 🚀 SFO 프로젝트 데이터 공통 전처리 
#
# - 원본 CSV 파일 로드부터 `df_clean` 생성까지의 공통 전처리
# - 생성된 df_clean을 csv로 저장
#
# [시작 전 준비사항]
# 1. `사출성형.csv` 파일 경로 확인 (input_csv_file)
# 2. 필요한 라이브러리 설치: `pip install pandas numpy scikit-learn ipykernel `
# 

import pandas as pd
import numpy as np
import os


In [ ]:

# --- 0. 설정 및 파일 경로 정의 ---
input_csv_file = '사출성형.csv'  
output_final_csv_file = 'data/baseline_processed.csv' # 전처리된 데이터 저장

target_column = 'T일 예정 수주량'


# --- 1단계: 데이터 로드 및 공통 전처리 (df_clean 생성) ---
print("\n--- [1단계: 데이터 로드 및 공통 전처리 시작] ---")
df_clean = None

try:
    if not os.path.exists(input_csv_file):
        raise FileNotFoundError(f"🚨 오류: 입력 파일 '{input_csv_file}'을(를) 찾을 수 없습니다. 경로를 확인해주세요.")

    df_raw = pd.read_csv(input_csv_file)
    print(f"1.1. 원본 CSV 파일 로드 완료 (shape: {df_raw.shape})")

    datetime_series = pd.to_datetime(df_raw['DateTime'], format='mixed', errors='coerce')
    df_raw['Date'] = datetime_series.dt.date
    df_raw['Time'] = datetime_series.dt.time

    # 다음 cell에서 숫자화해서 새로 만듦
    if 'DoW' in df_raw.columns:
        df_raw = df_raw.drop('DoW', axis=1)
        print("1.2. 'DoW' 컬럼 제거 완료.")
    

    if 'DateTime' in df_raw.columns:
        df_raw.drop(['DateTime'], axis=1, inplace=True)
        print("1.3. 'DateTime' 컬럼 제거 완료.")

    # 'Product_Number', 'Date', 'Time', 'T일 예정 수주량'을 기준으로 정렬
    df_raw.sort_values(by=['Product_Number', 'Date', 'Time', 'T일 예정 수주량'], inplace=True)
    print("1.4. 데이터 정렬 완료 (Product_Number, Date, Time).")

    # 마지막 값 남기고 제거
    data_df = df_raw.drop_duplicates(['Product_Number', 'Date'], keep='last').reset_index(drop=True)
    print(f"1.5. Product_Number-Date 중복 제거 완료 (shape: {data_df.shape}).")

    # 습도 이상치 처리(이상치-> 정상구간의 중앙값)
    col_humidity = 'Humidity'

    Q1_h = data_df[col_humidity].quantile(0.25)
    Q3_h = data_df[col_humidity].quantile(0.75)
    IQR_h = Q3_h - Q1_h
    lower_bound_h = Q1_h - 1.5 * IQR_h
    upper_bound_h = Q3_h + 1.5 * IQR_h

    non_outlier_values = data_df[ (data_df[col_humidity] >= lower_bound_h) & 
                                 (data_df[col_humidity] <= upper_bound_h)][col_humidity]

    median_h = non_outlier_values.median()

    data_df[col_humidity] = data_df[col_humidity].apply(
    lambda x: median_h if (x < lower_bound_h) or (x > upper_bound_h) else x)

    print(f"1.6 '{col_humidity}' 이상치 → 중앙값({median_h:.2f})으로 대체 완료 (shape: {data_df.shape})")
   
    # T-1, T-2, T-3,,, 만들기

    print("1.7 시계열 재배열 (T+N → T-N 형태로 정렬)")
    horizons = [1, 2, 3, 4]

    # 우리가 커버할 원본 컬럼 패턴들
    plan_sources = [
        ("T+{h}일 예정 수주량",        "T-{h}_예정"),     
        ("T+{h}일 예상 수주량",        "T-{h}_예상"),     
        ("작년 T+{h}일 예정 수주량",   "T-{h}_작년"), 
        
    ]

    for h in horizons:
        for src_tmpl, dst_tmpl in plan_sources:
            src_col = src_tmpl.format(h=h)
            dst_col = dst_tmpl.format(h=h)

            if src_col in data_df.columns:
                # h일 전 snapshot을 오늘 row에 붙인다
                data_df[dst_col] = data_df.groupby('Product_Number', group_keys=False)[src_col].shift(h)
            # 없으면 그냥 skip (컬럼이 실제로 없는 경우 대비)

    # 1.8 최종적으로 공유/저장할 df_clean 컬럼 선택
    final_cols = [
        'Product_Number',

        # 타깃 및 기준열
        'T일 예정 수주량',
        '작년 T일 예정 수주량',
        'T일 예상 수주량',

        # 재배열된 계획 스냅샷들
        'T-1_예정', 'T-1_예상', 'T-1_작년',
        'T-2_예정', 'T-2_예상', 'T-2_작년',
        'T-3_예정', 'T-3_예상', 'T-3_작년',
        'T-4_예정', 'T-4_예상', 'T-4_작년',

        # 환경 요인
        'Temperature', 'Humidity',

        # 날짜/시간
        'Date', 'Time',
    ]

    df_clean = data_df[final_cols].copy()

    # 1.9 Date를 pandas datetime으로 통일해서 저장 (나중에 merge/sort 안정성 ↑)
    # 지금 Date가 dtype=object (python date)일 가능성 → pandas Timestamp로 바꿔줌
    df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')


    # Time 컬럼 삭제
    if 'Time' in df_clean.columns:
        df_clean.drop(columns=['Time'], inplace=True)

    # NaN 있는 행 제거하고 싶으면 주석 제거 후 사용
    # df_clean = df_clean.dropna().reset_index(drop=True)

    df_clean.to_csv(output_final_csv_file, index=False, encoding='utf-8-sig')
    print(f"1단계 공통 전처리 완료! (df_clean shape: {df_clean.shape})")

except FileNotFoundError as e:
    print(f"🚨 {e}")
    df_clean = None
except Exception as e:
    print(f"🚨 1단계 데이터 로드 및 공통 전처리 중 오류 발생: {e}")
    df_clean = None




--- [1단계: 데이터 로드 및 공통 전처리 시작] ---
1.1. 원본 CSV 파일 로드 완료 (shape: (34617, 20))
1.2. 'DoW' 컬럼 제거 완료.
1.3. 'DateTime' 컬럼 제거 완료.
1.4. 데이터 정렬 완료 (Product_Number, Date, Time).
1.5. Product_Number-Date 중복 제거 완료 (shape: (10624, 20)).
1.6 'Humidity' 이상치 → 중앙값(28.03)으로 대체 완료 (shape: (10624, 20))
1.7 시계열 재배열 (T+N → T-N 형태로 정렬)
1단계 공통 전처리 완료! (df_clean shape: (10624, 19))


In [3]:
df_clean.columns

Index(['Product_Number', 'T일 예정 수주량', '작년 T일 예정 수주량', 'T일 예상 수주량', 'T-1_예정',
       'T-1_예상', 'T-1_작년', 'T-2_예정', 'T-2_예상', 'T-2_작년', 'T-3_예정', 'T-3_예상',
       'T-3_작년', 'T-4_예정', 'T-4_예상', 'T-4_작년', 'Temperature', 'Humidity',
       'Date'],
      dtype='object')

### ----머신러닝을 위한 Product_Number 라벨 인코딩 및 인코딩 파일 저장

In [4]:
from sklearn.preprocessing import LabelEncoder

le_product = LabelEncoder()
df_clean['Product_Number'] = le_product.fit_transform(df_clean['Product_Number'])


In [5]:
import json

mapping = dict(zip(le_product.classes_, le_product.transform(le_product.classes_)))
mapping = {k: int(v) for k, v in mapping.items()}  # <-- 핵심

with open('product_label_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(mapping, f, ensure_ascii=False, indent=2)
